Modelo de Dou et al. (2020)

Agora em Julia versão 1.5.3

using Pkg
Pkg.add("StatProfilerHTML")

In [50]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters

using BenchmarkTools

#packages to increase the speed of the code
using Profile # @profile command
using ProfileVega #profiling using graphs
# using StatProfilerHTML
using ProfileView #profiling using graphs
# using Traceur #analyses if your function has performance flaws such as changing the type of a variable


Parâmetros do Jogo

In [2]:
const μ = Float64(4.566) #número de meses entre períodos, traduz a quantidade de períodos t em quantidade de meses
const ρ = Float64(0.884) # (1 - ρ) é a taxa de depreciação da empresa a cada período
# const ρ = Float64(0.5) #testando nova taxa de depreciação
const β = Float64(9.84) #inverso da velocidade de aprendizado
const c0 = Float64(0.044) #custo fixo de ir para a corte
const c1 = Float64(0.015) #custo variável de ir para a corte


const hs0 = Float64(0.28) #habilidade inicial de s
const hj0 = Float64(0.36) #habilidade inicial de j

const λj = Float64(0.346)

#const is to alert Julia that these global variables will not change, so she does not have to worry about then

0.346

Valores que virão dos dados

In [3]:
const Vmax = Float64(1.0) #valor máximo de reorganização da firma
const L = Float64(0.25) #valor inicial de liquidação da firma
const Ds = Float64(0.32) #valor da dívida com credor sênior
const Dj = Float64(0.68) #valor da dívida com credor júnior

const D = Ds + Dj #valor total da dívida, usada para escalar custos

1.0

Definições iniciais: custos, valor máximo da firma, número de rounds

### Importante: vamos criar um jogo com T+1 períodos

Assim poderemos incluir o período zero nas matrizes

Então a notação do tempo vai ser tal que o período t é representado pelo índice t-1 nas funções, mas é representado por t nos vetores

In [4]:
#função custo
function Ct(t)
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 * D + c1 * (t-1) * D
    end
end

#precisa de um 'end' para o if e um 'end' para a função
    
#test
@show Ct(0)    
@show Ct(1)
@show Ct(2)
@show Ct(2) == c0 * D + c1 * (1) * D

#o fato de o custo ser função de D implica que duas empresas com mesmo valor de V e L podem ter destinos diferentes, pois
#a empresa com maior D terá maiores custos dentro da corte

Ct(0) = 0
Ct(1) = 0
Ct(2) = 0.059
Ct(2) == c0 * D + c1 * 1 * D = true


true

In [5]:
#liquidation payoffs

function s_L(t)
    return min(L - Ct(t), Ds)
end

@show s_L(0)
@show s_L(1)
@show s_L(2)


function j_L(t)
    return min(L - Ct(t) - s_L(t), Dj)
end
    
@show j_L(1)

s_L(0) = 0.25
s_L(1) = 0.25
s_L(2) = 0.191
j_L(1) = 0.0


0.0

In [6]:
#number of periods
function max_turns(Vmax, L, ρ)
    t = 0
    
    while(ρ^(t-1) * Vmax > L)
        t = t+1
    end
    return t
end


T = max_turns(Vmax, L, ρ)
T

13

In [7]:
#testando se T é o período final mesmo
@show t = T
@show ρ^(t-1) * Vmax - Ct(t) > L - Ct(t)


@show t = T-1
@show ρ^(t-1) * Vmax - Ct(t) > L - Ct(t)


#to see the values
@show t = T-1
@show ρ^(t-1) * Vmax - Ct(t)

@show L - Ct(t)

t = T = 13
ρ ^ (t - 1) * Vmax - Ct(t) > L - Ct(t) = false
t = T - 1 = 12
ρ ^ (t - 1) * Vmax - Ct(t) > L - Ct(t) = true
t = T - 1 = 12
ρ ^ (t - 1) * Vmax - Ct(t) = 0.04861723529489925
L - Ct(t) = 0.041000000000000036


0.041000000000000036

In [8]:
#função com valor máximo de reorganização da firma a cada período
function Vt(Vmax, Tmax, ρ)
    
    V = Array{Float64}(undef, Tmax+1)
    V[1] = Vmax
    
    #range exclui o valor máximo, daí que eu adicionei 1
    
    for t in 2:Tmax+1
        
        V[t] = Vmax * ρ^((t-1)-1) #(t-1) because the index starts at 1 in Julia
    end
    
    return V
end

V = Vt(Vmax, T, ρ)
V

#index 1 corresponds to t=0 and 2 to t=1...

14-element Array{Float64,1}:
 1.0
 1.0
 0.884
 0.781456
 0.690807104
 0.6106734799360001
 0.539835356263424
 0.4772144549368668
 0.4218575781641903
 0.37292209909714424
 0.3296631356018755
 0.29142221187205797
 0.2576172352948992
 0.2277336360006909

In [9]:
#array with costs each period

C = similar(V)

C[1] = 0

for t in 2:T+1
    C[t] = Ct(t)
end
    
C

14-element Array{Float64,1}:
 0.0
 0.059
 0.074
 0.089
 0.104
 0.119
 0.134
 0.149
 0.16399999999999998
 0.179
 0.194
 0.20899999999999996
 0.22399999999999998
 0.239

Skill levels' grid and Probability Mass Function(PMF)

Important: we won't use the skill levels directly in the functions, we will use their indexes. For example, index = 40 corresponds to $\theta_{K,t} = 0.4$.

In [10]:
#grid size
grid = 100

hlow = 0.01
hhigh = 1.0

#começa no menor valor possível, vai até o maior valor possível num intervalo do tamanho do grid
hvals = LinRange(hlow, hhigh, grid)

hvals
hvals[6]

0.06

In [11]:
#array to tell us the size of the pie at period t
#arguments: (period, θk,t)
U = Array{Float64}(undef,T+1, grid)


#cartesianIndices returns the index(i,j), and index(i,j)[1] = i, index(i,j)[2] = j
for i in CartesianIndices(U)
    U[i] = hvals[i[2]] * V[i[1]] - C[i[1]]
end  

#if t==0 (index 1), then there are no costs
@show U[1,7] == hvals[7] * V[1]

@show U[1, 7] == hvals[7] * Vmax

#if t>0 (index>1), there are costs
@show U[2,1] == hvals[1] * V[2] - Ct(2)

#checking the last period, T, whose index is "T+1"
#t = T+1
@show U[T+1,7] ==  hvals[7] * V[T+1] - Ct(T+1)

U[1, 7] == hvals[7] * V[1] = true
U[1, 7] == hvals[7] * Vmax = true
U[2, 1] == hvals[1] * V[2] - Ct(2) = true
U[T + 1, 7] == hvals[7] * V[T + 1] - Ct(T + 1) = true


true

In [12]:
#cumulative distribution function
function cdf(x, lt)
    
    #return the cdf of x given the lower bound lt
    #geq than 1 because of our discretization method 
    if(x >= 1)
        
        return 1
    
    else
        
        if(x >= lt)
            return 1 - ((1-x)^β)/((1-lt)^β)
        else
            return 0
        end
    end
end

#test      
@show cdf(0.5, 1)

@show cdf(0.5, 0.4)

@show cdf(0.99, 0.01)

@show cdf(1, 0.01)

@show cdf(1, 0.9)

cdf(0.5, 1) = 0
cdf(0.5, 0.4) = 0.8337136736331454
cdf(0.99, 0.01) = 1.0
cdf(1, 0.01) = 1
cdf(1, 0.9) = 1


1

In [13]:
#nova versão da função pmf: agora cada ponto do grid é o centro da cdf
#pmf: each point of the grid is at the center of the cdf
function pmf_cdf(lt, δ = hlow/2)
    pmf = zeros(grid)
    
    for (i, h) in enumerate(hvals)
        pmf[i] = cdf(h+δ, lt) - cdf(h-δ, lt)
    end
    
    return pmf
end


lt = 0.01
pmf_cdf(lt)

100-element Array{Float64,1}:
 0.048602089527521986
 0.09088953125821697
 0.08301186884522238
 0.07574579901540057
 0.06904947823155516
 0.06288353216214904
 0.05721093367839869
 0.05199688568380789
 0.04720870863310611
 0.04281573260029048
 0.038789193758253626
 0.03510213513518168
 0.03172931151561542
 ⋮
 3.7337122282821156e-10
 1.615731992643532e-10
 6.408462649432067e-11
 2.2833512858255745e-11
 7.107758825952715e-12
 1.856736986383112e-12
 3.83026943495679e-13
 5.651035195342047e-14
 4.9960036108132044e-15
 2.220446049250313e-16
 0.0
 0.0

In [14]:
#generating the pmfs

#probability mass function
pmf = Array{Float64}(undef,grid,grid)


for (t, θt) in enumerate(hvals)
    pmf[t,:] = pmf_cdf(θt)
end


#we need a pmf for when we have lkt and want to know θk,t+1
#we will call it 'pmf2'
pmf2 = similar(pmf)

for i in eachindex(hvals)
    for j in eachindex(hvals)
        pmf2[i, j] = sum(pmf[i, :] .*pmf[:, j])
    end
end

#rows are the skill levels today, columns are the skill levels tomorrow
pmf2[1,:] - pmf[1,:]

#should be nice to plot the two pmfs, so we will get the intuition easily

100-element Array{Float64,1}:
 -0.046239926421080726
 -0.08201062544335427
 -0.06652389304197329
 -0.05297653770265348
 -0.04117778693388258
 -0.03095216055945394
 -0.022138473358402036
 -0.014588889342129194
 -0.008168025651978328
 -0.0027521041142639507
  0.0017718514572064026
  0.0055067740688657385
  0.008546270271650315
  ⋮
  7.710965259700306e-9
  3.4895426000488317e-9
  1.4511143376026725e-9
  5.43807346649936e-10
  1.7875565752175066e-10
  4.95698020399729e-11
  1.093125330283789e-11
  1.7426855209729608e-12
  1.6978822815991093e-13
  7.239768851825965e-15
  5.665150746454088e-17
  8.758153619997799e-22

Arrays with the continuation values

In [15]:
#continuation values
#period t (not t+1), θkt, ℓkt, ℓmt
s_W = zeros(T+1, grid, grid, grid)
j_W = copy(s_W)


#optimal payments
#period, θkt, ℓmt, outputs
Pst_array = zeros(T, grid, grid, 6) #we won't need payment proposal for period T, a.k.a. "T+1"
Pjt_array = copy(Pst_array)

13×100×100×6 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.

In [16]:
#populating the last period with the liquidation values
s_W[T+1, :, :, :] .= s_L(T+1) #we need to use .= because it is a broadcast
j_W[T+1, :, :, :] .= j_L(T+1)

100×100×100 view(::Array{Float64,4}, 14, :, :, :) with eltype Float64:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  

In [17]:
#cutoff function: upgrade

function getcutoff_m(t, Pkt, hkt, m_W)

    #we extract the diagonal values, i.e., where hmt == lmt, to search for the cutoff
    diag_vals = @views diag(m_W[t+1, :, :, hkt])
    
    return searchsortedfirst(diag_vals, Pkt)
end

#should return 100 because J's continuation value at T is zero
@btime getcutoff_m(T, 0.01, 50, j_W)


  216.933 ns (1 allocation: 896 bytes)


101

In [18]:
#function to update the lowerbound according to the cutoff
#get the right lmt+1

function cutoff_m(t, Pkt, hkt, lmt, m_W)
    cmt = getcutoff_m(t, Pkt, hkt, m_W)
    
    if(cmt==101)
        return lmt
    else
        return max(cmt, lmt)
    end
end


@btime cutoff_m(T, 0.01, 50, 40, j_W)

#PAREI DE USAR ESSA FUNÇÃO NO TESTE COM LOOP

  219.663 ns (1 allocation: 896 bytes)


40

In [19]:
#function to select the pmf function as it is writen in the paper

function h_m(t, lmt)
    #takes the indexes of cutoff and lmt
        
    #at t=0(index 1), all information is symmetric, so we use pmf instead of pmf2mm
    if(t <= 1)
        return pmf[lmt, lmt:end]
    else
        return pmf2[lmt, lmt:end]
    end
end
    
#test
@show h_m(1, 50) == pmf[50, 50:end]

@show h_m(2, 50) == pmf2[50, 50:end]

@btime h_m(2,50)

h_m(1, 50) == pmf[50, 50:end] = true
h_m(2, 50) == pmf2[50, 50:end] = true
  144.523 ns (2 allocations: 528 bytes)


51-element Array{Float64,1}:
 0.008866568891510005
 0.03134054034293772
 0.053998301693260115
 0.06846396707993126
 0.07657685858255744
 0.07985914015113041
 0.0795594757138284
 0.07669193629092115
 0.07207054732437024
 0.06633984527073677
 0.06000179102641623
 0.05343936695638518
 0.046937164177679076
 ⋮
 4.625716199215591e-6
 2.128501871030038e-6
 8.99899204442307e-7
 3.428623147305904e-7
 1.1459581460290833e-7
 3.2321165203169804e-8
 7.253369111549412e-9
 1.1778647340830674e-9
 1.1710929608790348e-10
 5.110233142412765e-12
 4.3007226426965164e-14
 1.7849057398291997e-18

In [20]:
share = [0.0:hlow:hhigh;] #possible shares of the pie


#function to calculate the optimal payment offer
function Pkt(t, hkt, lmt, k_W, m_W, k_L)

    Pkt_grid = zeros(length(share) + 1)

    #the first possible value is -Vmax, which is an offer so low that will be rejected for sure (waiting offer)
    Pkt_grid[1] = -Vmax

    #the other values are the possible shares of the pie today
    #t+1 is just because we have T+1 periods in U, so U[1] is t==0 and U[T] is t==T-1
    #the reorganization payoff is U_{t} (\theta_{t+1})
    Pkt_grid[2:end] = U[t, hkt] * share

    #descartando os valores que são menores que o menor valor de continuação possível do adversário
    #assim não confundiremos uma proposta de reorganização com uma de espera
    lowest = m_W[t+1, lmt, lmt, hkt]

    Pkt_grid[2:end] = @. ifelse(Pkt_grid[2:end] < lowest, lowest, Pkt_grid[2:end])

    cutoffs = Array{Int64}(undef, length(Pkt_grid))

     #loop for calculating the cutoffs
    for (i, Pkt) in enumerate(Pkt_grid)
        cutoffs[i] = cutoff_m(t, Pkt, hkt, lmt, m_W)
    end

    #array com as probabilidades

    #probabilidade de m amanhã, dado o lower bound hoje ou o cutoff hoje

    #shape is Pkt x M
    probm = zeros(length(Pkt_grid), length(hvals[lmt:end]))

    for i in eachindex(Pkt_grid)
        probm[i, :] = h_m(t, lmt)
    end

    #probabiilidades de k amanhã dado a habilidade verdadeira hoje. 
    probk = pmf[hkt, hkt:end]

    #payoff if the proposal is accepted
    #pE1[3,6] is the third possible payment combined with the sixth hability above hkt
    # pE1 = np.array(U[t, None, find(hkt):] - Pkt_grid[:, None]) #python code

    pE1 = repeat(transpose(U[t, hkt:end]), length(Pkt_grid),1) .- Pkt_grid

    # pE1[10,50]

    #multiplicando pE1 pelas probabilides das habilidades futuras de k
    pE1 = pE1 .* repeat(transpose(probk), size(pE1)[1], 1)

    pE1 = sum(pE1, dims=2) #dims=2 is the dimension over which it will sum

    #ponderação pelas probabilidades de m
    # pE1 = np.multiply(pE1[:, None], probm)

    pE1 = repeat(pE1, 1, size(probm)[2]) .* probm

    #indicator array
    cont_vals = zeros(length(Pkt_grid), length(hvals[lmt:end]))

    # cont_vals = Array{Float64}(undef, length(Pkt_grid), length(hvals[find(lmt):end]))

    #continuation values separated before and after screening cutoffs

    #if cutoffs > lmt, then we use lmt+1 = lmt to all theta below cutoffs and lmt+1 = cutoffs to all theta above cutoffs
    #in julia, range(a:b) includes b. it's different from python
    for i in eachindex(Pkt_grid)
        if(cutoffs[i] > lmt)#there is a lowerbound update
            cont_vals[i, 1:(cutoffs[i]-lmt)] .= m_W[t+1, lmt:(cutoffs[i]-1), lmt, hkt]
            cont_vals[i, (cutoffs[i]-lmt+1):end] .= m_W[t+1, cutoffs[i]:end, cutoffs[i], hkt]
        else
            cont_vals[i, 1:end] .= m_W[t+1, lmt:end, lmt, hkt]
        end
    end

    #array indicating if the possible continuation values of the adversary are leq than each payment offer

    # IE1 = np.where(cont_vals <= Pkt_grid[:,None] , 1, 0) #python code
    IE1 = @. ifelse(cont_vals <= Pkt_grid, 1,0)

    #payoff E1
    E1 = pE1 .* IE1

    #we sum along the columns (dims=2) so as to each row contain the expected payoff of proposing a payment offer
    E1 = sum(E1, dims=2)

    # #cálculo de E2####

    #E2 payoff can use the cutoffs directly, i.e., doesn't need to separate before and after cutoff
    #this is because pE2 will only happen if M declines the offer, which only happens if his ability is above the cutoff

    pE2 = k_W[t+1, hkt:end, hkt, cutoffs]

    #multiplying pE2 by the probabilities
    #multiplicando pE2 pelo array de probabilidades
    # pE2 = np.multiply(pE2, probk[None, :])
    # pE2 = np.sum(pE2, axis = 1)

    pE2 = pE2 .* repeat(probk,1 , size(pE2)[2])

    pE2 = sum(pE2, dims=1) #dims=2 is the dimension over which it will sum

    #pE2 = np.multiply(pE2[:,None], probm) #python code

    pE2 = repeat(transpose(pE2), 1, size(probm)[2]) .* probm

    #indicator array
    # IE2 = np.where(IE1 == 1, 0, 1) #python code


    IE2 = @. ifelse(IE1 ==1, 0,1)

    #payoff

    E2 = pE2 .* IE2

    E2 = sum(E2, dims=2)

    #mpayoffs matrix####
    matrix_payoff = vec(E1 + E2) #vec is to convert it to a single column-matrix

    payoff_reorg, index_reorg = findmax(matrix_payoff)

    payment = Pkt_grid[index_reorg]

    #calculating the optimal policy between liquidating, reorganizing or waiting ####
    payoff_liq = k_L(t)

    #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
    payoff_wait = matrix_payoff[1]

    payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))

    #returns the expected payoff of a waiting proposal to make debugging easier
    return payment, cutoffs[index_reorg], payoff_reorg, payoff_wait, payoff_max, policy
    
end


#testing hk = 0.5
t, hkt, lmt, k_W, m_W, k_L = T, 50, 60, s_W, j_W, s_L
@show Pkt(t, hkt, lmt, k_W, m_W, k_L)

#testing hk = 1
t, hkt, lmt, k_W, m_W, k_L = T, 100, 60, s_W, j_W, s_L
@show Pkt(t, hkt, lmt, k_W, m_W, k_L)


@btime Pkt(t, hkt, lmt, k_W, m_W, k_L)

Pkt(t, hkt, lmt, k_W, m_W, k_L) = (-1.0, 60, 0.011000000000000038, 0.011000000000000038, 0.026000000000000023, 1)
Pkt(t, hkt, lmt, k_W, m_W, k_L) = (0.0, 60, 0.03361723529489927, 0.011000000000000022, 0.03361723529489927, 3)
  239.999 μs (1962 allocations: 573.97 KiB)


(0.0, 60, 0.03361723529489927, 0.011000000000000022, 0.03361723529489927, 3)

In [21]:
U[T, 100]

0.03361723529489924

In [74]:
s_L(T)

0.026000000000000023

In [158]:
share = [0.0:hlow:hhigh;] #possible shares of the pie


#function to calculate the optimal payment offer

#version that directly populates the array
function Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array)

    Pkt_grid = zeros(length(share) + 1)

    #the first possible value is -Vmax, which is an offer so low that will be rejected for sure (waiting offer)
    Pkt_grid[1] = -Vmax

    #the other values are the possible shares of the pie today
    #t+1 is just because we have T+1 periods in U, so U[1] is t==0 and U[T] is t==T-1
    #the reorganization payoff is U_{t} (\theta_{t+1})
    Pkt_grid[2:end] = U[t, hkt] * share

    #descartando os valores que são menores que o menor valor de continuação possível do adversário
    #assim não confundiremos uma proposta de reorganização com uma de espera
    lowest = m_W[t+1, lmt, lmt, hkt]

    Pkt_grid[2:end] = @. ifelse(Pkt_grid[2:end] < lowest, lowest, Pkt_grid[2:end])

    cutoffs = Array{Int64}(undef, length(Pkt_grid))

     #loop for calculating the cutoffs
    for (i, Pkt) in enumerate(Pkt_grid)
        cutoffs[i] = cutoff_m(t, Pkt, hkt, lmt, m_W)
    end

    #array com as probabilidades

    #probabilidade de m amanhã, dado o lower bound hoje ou o cutoff hoje

    #shape is Pkt x M
    probm = zeros(length(Pkt_grid), length(hvals[lmt:end]))

    for i in eachindex(Pkt_grid)
        probm[i, :] = h_m(t, lmt)
    end

    #probabiilidades de k amanhã dado a habilidade verdadeira hoje. 
    probk = pmf[hkt, hkt:end]

    #payoff if the proposal is accepted
    #pE1[3,6] is the third possible payment combined with the sixth hability above hkt
    # pE1 = np.array(U[t, None, find(hkt):] - Pkt_grid[:, None]) #python code

    pE1 = repeat(transpose(U[t, hkt:end]), length(Pkt_grid),1) .- Pkt_grid

    # pE1[10,50]

    #multiplicando pE1 pelas probabilides das habilidades futuras de k
    pE1 = pE1 .* repeat(transpose(probk), size(pE1)[1], 1)

    pE1 = sum(pE1, dims=2) #dims=2 is the dimension over which it will sum

    #ponderação pelas probabilidades de m
    # pE1 = np.multiply(pE1[:, None], probm)

    pE1 = repeat(pE1, 1, size(probm)[2]) .* probm

    #indicator array
    cont_vals = zeros(length(Pkt_grid), length(hvals[lmt:end]))

    # cont_vals = Array{Float64}(undef, length(Pkt_grid), length(hvals[find(lmt):end]))

    #continuation values separated before and after screening cutoffs

    #if cutoffs > lmt, then we use lmt+1 = lmt to all theta below cutoffs and lmt+1 = cutoffs to all theta above cutoffs
    #in julia, range(a:b) includes b. it's different from python
    for i in eachindex(Pkt_grid)
        if(cutoffs[i] > lmt)#there is a lowerbound update
            cont_vals[i, 1:(cutoffs[i]-lmt)] .= m_W[t+1, lmt:(cutoffs[i]-1), lmt, hkt]
            cont_vals[i, (cutoffs[i]-lmt+1):end] .= m_W[t+1, cutoffs[i]:end, cutoffs[i], hkt]
        else
            cont_vals[i, 1:end] .= m_W[t+1, lmt:end, lmt, hkt]
        end
    end

    #array indicating if the possible continuation values of the adversary are leq than each payment offer

    # IE1 = np.where(cont_vals <= Pkt_grid[:,None] , 1, 0) #python code
    IE1 = @. ifelse(cont_vals <= Pkt_grid, 1,0)

    #payoff E1
    E1 = pE1 .* IE1

    #we sum along the columns (dims=2) so as to each row contain the expected payoff of proposing a payment offer
    E1 = sum(E1, dims=2)

    # #cálculo de E2####

    #E2 payoff can use the cutoffs directly, i.e., doesn't need to separate before and after cutoff
    #this is because pE2 will only happen if M declines the offer, which only happens if his ability is above the cutoff

    pE2 = k_W[t+1, hkt:end, hkt, cutoffs]

    #multiplying pE2 by the probabilities
    #multiplicando pE2 pelo array de probabilidades
    # pE2 = np.multiply(pE2, probk[None, :])
    # pE2 = np.sum(pE2, axis = 1)

    pE2 = pE2 .* repeat(probk,1 , size(pE2)[2])

    pE2 = sum(pE2, dims=1) #dims=2 is the dimension over which it will sum

    #pE2 = np.multiply(pE2[:,None], probm) #python code

    pE2 = repeat(transpose(pE2), 1, size(probm)[2]) .* probm

    #indicator array
    # IE2 = np.where(IE1 == 1, 0, 1) #python code


    IE2 = @. ifelse(IE1 ==1, 0,1)

    #payoff

    E2 = pE2 .* IE2

    E2 = sum(E2, dims=2)

    #mpayoffs matrix####
    matrix_payoff = vec(E1 + E2) #vec is to convert it to a single column-matrix

    payoff_reorg, index_reorg = findmax(matrix_payoff)

    payment = Pkt_grid[index_reorg]

    #calculating the optimal policy between liquidating, reorganizing or waiting ####
    payoff_liq = k_L(t)

    #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
    payoff_wait = matrix_payoff[1]

    payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))

    #returns the expected payoff of a waiting proposal to make debugging easier
    Pkt_array[t, hkt, lmt, :] .= payment, cutoffs[index_reorg], payoff_reorg, payoff_wait, payoff_max, policy
    
end




#testing hk = 1
t, hkt, lmt, k_W, m_W, k_L, Pkt_array = T, 100, 60, s_W, j_W, s_L, Pst_array
@show Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array)

#testing hk = 0.5
t, hkt, lmt, k_W, m_W, k_L, Pkt_array = T, 50, 60, s_W, j_W, s_L, Pst_array
@show Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array)

@btime Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array)


Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array) = [0.0, 60.0, 0.03361723529489927, 0.011000000000000022, 0.03361723529489927, 3.0]
Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array) = [-1.0, 60.0, 0.011000000000000038, 0.011000000000000038, 0.026000000000000023, 1.0]
  307.600 μs (1991 allocations: 854.88 KiB)


6-element view(::Array{Float64,4}, 13, 50, 60, :) with eltype Float64:
 -1.0
 60.0
  0.011000000000000038
  0.011000000000000038
  0.026000000000000023
  1.0

Tentando acelerar Pkt


In [24]:


function Pkt2(t, hkt, lmt, k_W, m_W, k_L, Pkt_array)
    
    Pkt_grid = zeros(length(share))

    #the reorganization payoff is U_{t} (\theta_{t+1})
    Pkt_grid[:] .= U[t, hkt] * share


    cutoffs = Array{Int64}(undef, length(Pkt_grid))

     #loop for calculating the cutoffs
    for (i, Pkt) in enumerate(Pkt_grid)
        cutoffs[i] = getcutoff_m(t, Pkt, hkt, m_W)
    end

    #array com as probabilidades

    #probabilidade de m amanhã, dado o lower bound hoje ou o cutoff hoje

    #shape is Pkt x M
    probm = zeros(length(Pkt_grid), length(hvals[lmt:end]))

    for i in eachindex(Pkt_grid)
        probm[i, :] .= h_m(t, lmt)
    end


    #probabiilidades de k amanhã dado a habilidade verdadeira hoje. 
    probk = pmf[hkt, hkt:end]

    hk_vals = hkt:grid
    hm_vals = lmt:grid


    #parte realmente nova do código####
    payoff = Array{Float64}(undef, length(Pkt_grid))




    for (p, pkt) in enumerate(Pkt_grid)
        for (i, hm) in enumerate(hm_vals)
            if(hm < cutoffs[p])#it means M will accept the payment offer
                for (j, hk) in enumerate(hk_vals)
                    payoff[p] += (U[t, hk] - pkt) * probm[p, i] * probk[j]
                end

            else #when M rejects the payment offer
                for (j, hk) in enumerate(hk_vals)
                    payoff[p] += (k_W[t+1, hk, hkt, cutoffs[p]]) * probm[p, i] * probk[j]
                end


            end

        end

    end
    
    return findmax(vec(payoff))
end
            
            
t, hkt, lmt, k_W, m_W, k_L, Pkt_array = T, 50, 60, s_W, j_W, s_L, Pst_array
@btime Pkt2(t, hkt, lmt, k_W, m_W, k_L, Pkt_array)

#depois teremos que descartar as propostas de pagamento inferiores ao menor valor de continuação de M
#para não perder tempo ou confundir com propostas de esperar

  70.826 ms (1919323 allocations: 42.68 MiB)


(0.07699999999999972, 101)

In [ ]:
using Profile
for i in 1:100
    @profile Pkt2(t, hkt, lmt, k_W, m_W, k_L, Pkt_array)
end


Profile.print()

In [25]:
#function to calculate the waiting payoff
function payoff_wait(t, hkt, lmt, k_W)
    
    return dot(pmf[hkt, hkt:end], k_W[t+1, hkt:end, hkt, lmt]) 
    
end


@btime payoff_wait(t, hkt, lmt, k_W)
    
    

  322.364 ns (4 allocations: 1.02 KiB)


0.011000000000000018

In [248]:
function Pst(t, θst, ℓjt)
    return Pkt(t, θst, ℓjt, s_W, j_W, s_L, Pst_array)
end


function Pjt(t, θjt, ℓst)
    return Pkt(t, θjt, ℓst, j_W, s_W, j_L, Pjt_array)
end

Pjt (generic function with 1 method)

In [31]:
#populating the arrays to continue the tests
#we populate period T-1, whose index is T
t = T
for h in 1:grid
    for l in 1:grid
            Pst(t, h, l)
            Pjt(t, h, l)
    end
end

In [36]:
#optimal proposal
function propose(t, hkt, lkt, lmt, Pkt_array)
    #it just searches for the corresponding Pkt
    
    (payoff_max, policy) = @views Pkt_array[t, hkt, lmt, end-1:end]
    
    return payoff_max, policy
end

#test with hk = 0.5 (index 50)
t, hkt, lkt, lmt, Pkt_array = T, 50, 1, 60, Pst_array
@show propose(t, hkt, lkt, lmt, Pkt_array)

#test with hk = 1.0 (index 100)
hkt = 100
@show propose(t, hkt, lkt, lmt, Pkt_array)

@btime propose(t, hkt, lkt, lmt, Pkt_array)

propose(t, hkt, lkt, lmt, Pkt_array) = (0.026000000000000023, 1.0)
propose(t, hkt, lkt, lmt, Pkt_array) = (0.03361723529489927, 3.0)
  37.499 ns (1 allocation: 32 bytes)


(0.03361723529489927, 3.0)

In [140]:
#"cdf" for the discretized distribution

function cdf_discrete(x, lt)
    #returns the probability that θt takes a value less than x, given lt
    #we use strictly less than x because of the payment offer threshold
    return sum(pmf[lt, 1:(x-1)])
end

#test
# @show cdf(0.5, 0.01)
@show cdf(50, 1)

# @show cdf_discrete(0.5, 0.01)
@show cdf_discrete(50, 1)

@btime cdf_discrete(50,1)

cdf(50, 1) = 1
cdf_discrete(50, 1) = 0.998671575594218
  160.437 ns (3 allocations: 544 bytes)


0.998671575594218

In [141]:
function threshold_m(t, lkt, lmt, Pmt_array)
    
    #searches the index where the optimal policy is different from 1.0 (policy of liquidating the firm)
    threshold_vector = @views findall(Pmt_array[t, 1:end, lkt, end] .!= 1.0) #the dot before != is to broadcast
    
    #gives error message is threshold does not exist
    @assert size(threshold_vector)[1] > 0
    
    ts = @views minimum(threshold_vector)

    #probability that m will propose liquidation next period
    
    #at t==0 (index 1), the info is symmetric, so lmt == θmt, thus it becomes the indicator function
    if(t<=1)
        if(lmt >= ts)
            probm_liq = 0.0
        else
            probm_liq = 1.0
        end


    else#if t>0(index>1)

        if(lmt >= ts)
            probm_liq = 0.0
        else
            probm_liq = cdf_discrete(ts,lmt)
        end

    end
    
    #returns the reorganization threshold and the probability that m will propose liquidation in t
    return ts, probm_liq
end

#test 
t, lkt,lmt, Pmt_array = T, 50, 51, Pst_array
@btime threshold_m(t, lkt,lmt,  Pmt_array)

  612.797 ns (7 allocations: 1.19 KiB)


(97, 0.9999999999947272)

In [142]:
#function to calculate the payment offer threshold ϕmt

#test with searchsorted first

function threshold_m(t, lkt, lmt, Pmt_array)
    
    #searches the index where the optimal policy is different from 1.0 (policy of liquidating the firm)
    ts = @views searchsortedlast(Pmt_array[t, :, lkt, end], 1.0)+1
    
    
    #gives error message is threshold does not exist (101 is returned if missing by ssorted, plus 1)
    @assert ts < 101
    
    #probability that m will propose liquidation next period
    
    #at t==0 (index 1), the info is symmetric, so lmt == θmt, thus it becomes the indicator function
    if(t<=1)
        if(lmt >= ts)
            probm_liq = 0.0
        else
            probm_liq = 1.0
        end


    else#if t>0(index>1)

        if(lmt >= ts)
            probm_liq = 0.0
        else
            probm_liq = cdf_discrete(ts,lmt)
        end

    end
    
    #returns the reorganization threshold and the probability that m will propose liquidation in t
    return ts, Float64(probm_liq)
end

#test 
t, lkt,lmt, Pmt_array = T, 50, 51, Pst_array
@btime threshold_m(t, lkt,lmt,  Pmt_array)

  456.858 ns (4 allocations: 976 bytes)


(97, 0.9999999999947272)

In [103]:
#expected payoff of answering a payment offer2

function respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W)

    ts, probm_liq = threshold_m(t, lkt, lmt, Pmt_array)

    lmt = max(ts, lmt)


    #the possible values of payment given by the opponent
    Pmt_grid = Pmt_array[t, lmt:end, lkt, 1]
    cutoffs = Pmt_array[t, lmt:end, lkt, 2]

    cutoffs = convert(Array{Int64}, cutoffs)
    
    

    #A1### 

    #payoff when accepting the payment is the best response
    pA1 = copy(Pmt_grid)

    #weighting by the probabilities
    probk = pmf[hkt, hkt:end]
    probm = pmf[lmt, lmt:end]

    lowerbounds = collect(lmt:1:grid)
    
    
    #K and M to make the shape of the arrays easier to see
    K, M = length(probk), length(probm)


    pA2 = Array{Float64}(undef, K, M)
    for i in eachindex(cutoffs)
        pA2[:, i] .= (k_W[t+1, hkt:end, cutoffs[i], lowerbounds[i]])
    end

    pA2

    #indicator function
    # IA1 = np.where(k_W[t+1, hkt:, cutoffs, np.arange(lmt, grid, step = 1)].T <= Pmt_grid, 1, 0) #python code

    IA1 = Array{Int64}(undef, K, M)
    IA1[findall(pA2 .<= transpose(repeat(Pmt_grid, 1, K)))] .= 1

    #IA1 has S rows and J columns
    #we want a matrix of probabilities with the same dimension
    # prob = np.outer(probk, probm) #python code
#     prob = repeat(probk, 1, M) .* transpose(repeat(probm, 1, K))
    
    prob = probm' .* probk #outer product


    pA1 = transpose(pA1) .* prob

    A1 = pA1 .* IA1

    #A2####

    IA2 = @. ifelse(IA1==1, 0, 1)

    pA2 = pA2 .* prob

    A2 = pA2 .* IA2
    
    payoff = A1 + A2
    payoff = sum(payoff)

    #returns the expected value of answering a payment offer and the probability of liquidation
    return payoff * (1.0 - probm_liq), probm_liq
end


#test1: expected payoff of S
t, hkt, lkt, lmt, Pmt_array, k_W = T, 50, 50, 50, Pjt_array, s_W
@show respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W)

@btime respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W)

#test2: expected payoff of J
Pmt_array, k_W = Pst_array, j_W
@show respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W)

@btime respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W)


respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W) = (8.796868795624103e-10, 0.9999999200284655)
  38.599 μs (345 allocations: 70.02 KiB)
respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W) = (0.0, 0.999999999995678)


base64 binary data: CihqdWxpYS5leGU6NzcyOCk6IEd0ay1XQVJOSU5HICoqOiAxOToxNToyNC44NzA6IEVycm9yIGxvYWRpbmcgdGhlbWUgaWNvbiAnZG9jdW1lbnQtb3BlbicgZm9yIHN0b2NrOiBGYWlsZWQgdG8gbG9hZCBDOlxVc2Vyc1xa6VwuanVsaWFcYXJ0aWZhY3RzXDY1ZWNhN2M0OGRlYTFlMzIyMDNiMjA1NjEzNDQxY2U5NTA2MDQ1YjRcc2hhcmVcaWNvbnNcQWR3YWl0YVwyNHgyNC9sZWdhY3lcZG9jdW1lbnQtb3Blbi5wbmc6IFVucmVjb2duaXplZCBpbWFnZSBmaWxlIGZvcm1hdAoKKGp1bGlhLmV4ZTo3NzI4KTogR3RrLVdBUk5JTkcgKio6IDE5OjE1OjI0Ljg3MDogRXJyb3IgbG9hZGluZyB0aGVtZSBpY29uICdpbWFnZS1taXNzaW5nJyBmb3Igc3RvY2s6IEZhaWxlZCB0byBsb2FkIEM6XFVzZXJzXFrpXC5qdWxpYVxhcnRpZmFjdHNcNjVlY2E3YzQ4ZGVhMWUzMjIwM2IyMDU2MTM0NDFjZTk1MDYwNDViNFxzaGFyZVxpY29uc1xBZHdhaXRhXDI0eDI0L3N0YXR1c1xpbWFnZS1taXNzaW5nLnBuZzogVW5yZWNvZ25pemVkIGltYWdlIGZpbGUgZm9ybWF0CgooanVsaWEuZXhlOjc3MjgpOiBHZGstQ1JJVElDQUwgKio6IDE5OjE1OjI0Ljg3MDogZ2RrX2NhaXJvX3N1cmZhY2VfY3JlYXRlX2Zyb21fcGl4YnVmOiBhc3NlcnRpb24gJ0dES19JU19QSVhCVUYgKHBpeGJ1ZiknIGZhaWxlZAoKKGp1bGlhLmV4ZTo3NzI4KTogR0xpYi1HT2JqZWN0LUNSSVRJQ0FMICoqOiAxOToxNToyNC44NzA6IGdfb2JqZWN0

  18.500 μs (164 allocations: 29.13 KiB)


(0.0, 0.999999999995678)

# Tentando acelerar respond_payment

separando função threshold em duas funções diferentes

In [222]:
#function to calculate the payment offer threshold ϕmt

#test with searchsorted first

function threshold_m(t, lkt, lmt, Pmt_array)
    
    #searches the index where the optimal policy is either 2.0 or 3.0 (wait or reorganize)
    #"Return the index of the first value in a greater than or equal to x, according to the specified order. 
    #Return length(a) + 1 if x is greater than all values in a. a is assumed to be sorted."
    ts = @views searchsortedfirst(Pmt_array[t, :, lkt, end], 2.0)
    
    
    #gives error message is threshold does not exist (101 is returned if missing by ssorted, plus 1)
    @assert ts < 101
    
    return ts
    
end

#test 
t, lkt,lmt, Pmt_array = T, 50, 51, Pst_array
@btime threshold_m(t, lkt,lmt,  Pmt_array)

  43.774 ns (0 allocations: 0 bytes)


97

In [223]:
function prob_liq_m(t, ts, lmt)::Float64
    
    #probability that m will propose liquidation next period
    
    #at t==0 (index 1), the info is symmetric, so lmt == θmt, thus it becomes the indicator function
    if(t<=1)
        if(lmt >= ts)
            return 0.0
        else
            return 1.0
        end


    else#if t>0(index>1)

        if(lmt >= ts)
            return 0.0
        else
            return cdf_discrete(ts,lmt)
        end

    end
    
    
end

t, lkt,lmt, Pmt_array = T, 50, 51, Pst_array
ts = threshold_m(t, lkt,lmt,  Pmt_array)
@btime prob_liq_m(t, ts, lmt )
    

  302.873 ns (4 allocations: 960 bytes)


0.9999999999947272

In [156]:
#test1: expected payoff of S
t, hkt, lkt, lmt, Pmt_array, k_W = T, 50, 50, 50, Pjt_array, s_W


ts = threshold_m(t, lkt, lmt, Pmt_array)

lmt = max(ts, lmt)


#the possible values of payment given by the opponent
Pmt_grid = Pmt_array[t, lmt:end, lkt, 1]
cutoffs = Pmt_array[t, lmt:end, lkt, 2]


cutoffs

# cutoffs = convert(Array{Int64}, cutoffs)

# hk_vals = hkt:grid
# hm_vals = lmt:grid


# payoff = zeros(length(Pmt_grid))


# i = 1
# k_W[t+1, hkt, cutoffs[i]:end, hm_vals[i]]

# for (i, pmt) in enumerate(Pmt_grid)
#     payoff[i] += pmt * sum(pmf[hkt, hkt:(cutoffs[i]-1)]) * pmf[lmt, hm_vals[i]]
#     payoff[i] += dot(k_W[t+1, cutoffs[i]:end, cutoffs[i], hm_vals[i]], pmf[hkt, cutoffs[i]:end]) * pmf[lmt, hm_vals[i]]
# end

    
# for (i, pmt) in enumerate(Pmt_grid)
#     for (j, hk) in enumerate(hk_vals)
#         if(hk < cutoffs[i]) #means it is better to accept
#             payoff[i] += pmt * pmf[hkt, hk] * pmf[lmt, hm_vals[i]]
#         else
#             payoff[i] += k_W[t+1, hk, cutoffs[i], hm_vals[i]] * pmf[hkt, hk] * pmf[lmt, hm_vals[i]]
            
#         end
#     end
# end


# sum(payoff) * (1-probm_liq), probm_liq




# k_W[t+1, hkt:end, cutoffs, lmt:end]

10-element Array{Float64,1}:
 50.0
 50.0
 50.0
 50.0
 50.0
 50.0
 50.0
 50.0
 50.0
 50.0

In [224]:


function respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W)
    
    ts = threshold_m(t, lkt, lmt, Pmt_array)
    probm_liq = prob_liq_m(t, ts, lmt)

    lmt = max(ts, lmt)


    #the possible values of payment given by the opponent
    Pmt_grid = Pmt_array[t, lmt:end, lkt, 1]
    cutoffs = convert(Array{Int64}, Pmt_array[t, lmt:end, lkt, 2])


    payoff = zeros(length(Pmt_grid))



    @inbounds for (i, pmt) in enumerate(Pmt_grid)
        payoff[i] += pmt * sum(pmf[hkt, hkt:(cutoffs[i]-1)]) * pmf[lmt, lmt+(i-1)]
        payoff[i] += @views dot(k_W[t+1, cutoffs[i]:end , cutoffs[i], lmt+(i-1)], pmf[hkt, cutoffs[i]:end]) * pmf[lmt, lmt+(i-1)]
        
    end
        
    return sum(payoff) * (1.0-probm_liq), probm_liq
    
end


#test1: expected payoff of S
t, hkt, lkt, lmt, Pmt_array, k_W = T, 50, 50, 50, Pjt_array, s_W
@btime respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W)

  12.399 μs (228 allocations: 7.44 KiB)


(8.7968687956241e-10, 0.9999999200284655)

In [201]:
#test2: expected payoff of J
Pmt_array, k_W = Pst_array, j_W
@btime respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W)

  5.550 μs (96 allocations: 3.77 KiB)


(0.0, 0.999999999995678)

In [108]:
Profile.clear()
for i in 1:10000
    @profile respond_payment2(t, hkt, lkt, lmt, Pmt_array, k_W)
end

ProfileView.view()

base64 binary data: CihqdWxpYS5leGU6NzcyOCk6IEd0ay1XQVJOSU5HICoqOiAxOToxNzoyNC4zMTc6IEVycm9yIGxvYWRpbmcgdGhlbWUgaWNvbiAnZG9jdW1lbnQtb3BlbicgZm9yIHN0b2NrOiBGYWlsZWQgdG8gbG9hZCBDOlxVc2Vyc1xa6VwuanVsaWFcYXJ0aWZhY3RzXDY1ZWNhN2M0OGRlYTFlMzIyMDNiMjA1NjEzNDQxY2U5NTA2MDQ1YjRcc2hhcmVcaWNvbnNcQWR3YWl0YVwyNHgyNC9sZWdhY3lcZG9jdW1lbnQtb3Blbi5wbmc6IFVucmVjb2duaXplZCBpbWFnZSBmaWxlIGZvcm1hdAoKKGp1bGlhLmV4ZTo3NzI4KTogR3RrLVdBUk5JTkcgKio6IDE5OjE3OjI0LjMxNzogRXJyb3IgbG9hZGluZyB0aGVtZSBpY29uICdpbWFnZS1taXNzaW5nJyBmb3Igc3RvY2s6IEZhaWxlZCB0byBsb2FkIEM6XFVzZXJzXFrpXC5qdWxpYVxhcnRpZmFjdHNcNjVlY2E3YzQ4ZGVhMWUzMjIwM2IyMDU2MTM0NDFjZTk1MDYwNDViNFxzaGFyZVxpY29uc1xBZHdhaXRhXDI0eDI0L3N0YXR1c1xpbWFnZS1taXNzaW5nLnBuZzogVW5yZWNvZ25pemVkIGltYWdlIGZpbGUgZm9ybWF0CgooanVsaWEuZXhlOjc3MjgpOiBHZGstQ1JJVElDQUwgKio6IDE5OjE3OjI0LjMxNzogZ2RrX2NhaXJvX3N1cmZhY2VfY3JlYXRlX2Zyb21fcGl4YnVmOiBhc3NlcnRpb24gJ0dES19JU19QSVhCVUYgKHBpeGJ1ZiknIGZhaWxlZAoKKGp1bGlhLmV4ZTo3NzI4KTogR0xpYi1HT2JqZWN0LUNSSVRJQ0FMICoqOiAxOToxNzoyNC4zMTc6IGdfb2JqZWN0

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1,000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint

In [225]:
#expected payoff of answering a liquidation offer

function respond_liq(t, hkt, lkt, lmt, k_L, m_L)
    
    #payoff_reorg is the payoff of reorganizing while paying the opponent's liquidation value
    #U_{t} (\theta_{t+1}) because the skill level is of the next period
    #dot is matrix multiplication
    payoff_reorg = dot(pmf[hkt, hkt:end], (U[t, hkt:end] .- m_L(t)))
    
    payoff_liq = k_L(t)
    
    return max(payoff_liq, payoff_reorg)
end


#what happens if J proposes to liquidation at the penultimate period and S answers it?
#hkt = 0.5
t, hkt, lkt, lmt, k_L, m_L = T, 50, 50, 50, s_L, j_L
@show respond_liq(t, hkt, lkt, lmt, k_L, m_L)


#changing just hkt
#hkt = 1.0
hkt = 100
@show respond_liq(t, hkt, lkt, lmt, k_L, m_L)
@show dot(pmf[hkt, hkt:end], (U[t, hkt:end] .- m_L(t)))


@btime respond_liq(t, hkt, lkt, lmt, k_L, m_L)

respond_liq(t, hkt, lkt, lmt, k_L, m_L) = 0.026000000000000023
respond_liq(t, hkt, lkt, lmt, k_L, m_L) = 0.03361723529489924
dot(pmf[hkt, hkt:end], U[t, hkt:end] .- m_L(t)) = 0.03361723529489924
  722.143 ns (11 allocations: 480 bytes)


0.03361723529489924

In [226]:
#expected payoff of being called to respond
function respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L)
    
    payoff_rpayment, probm_liq = respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W)
    
    return payoff_rpayment + probm_liq * respond_liq(t, hkt, lkt, lmt, k_L, m_L)
end

#test
t, hkt, lkt, lmt, k_L, m_L, Pmt_array, k_W = T, 50, 50, 50, s_L, j_L, Pst_array, j_W

@show respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L)


respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L) = 0.02599999999988765


0.02599999999988765

In [250]:
#defining S functions based on the generic functions

#Pkt(t, hkt, lmt, k_W, m_W, k_L)
function Pst(t, θst, ℓjt)
    return Pkt(t, θst, ℓjt, s_W, j_W, s_L, Pst_array)
end

#propose(t, hkt, lkt, lmt, Pkt_array)
function s_propose(t, θst, ℓst, ℓjt)
    return propose(t, θst, ℓst, ℓjt, Pst_array)
end

#threshold_m(t, lkt, lmt, Pmt_array)
function threshold_j(t, ℓst, ℓjt)
    return threshold_m(t, ℓst, ℓjt, Pjt_array)
end


#respond_reorg(t, hkt, lkt, lmt, Pmt_array, k_W)
function s_respond_payment(t, θst, ℓst, ℓjt)
    return respond_payment(t, θst, ℓst, ℓjt, Pjt_array, s_W)
end


#respond_liq(t, hkt, lkt, lmt, k_L, m_L)
function s_respond_liq(t, θst, ℓst, ℓjt)
    return respond_liq(t, θst, ℓst, ℓjt, s_L, j_L)
end

    
#respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L) 
function s_respond(t, θst, ℓst, ℓjt)
    return respond(t, θst, ℓst, ℓjt, Pjt_array, s_W, s_L, j_L)
end

s_respond (generic function with 1 method)

In [251]:
#J's functions based on the generic functions
function Pjt(t, θjt, ℓst)
    return Pkt(t, θjt, ℓst, j_W, s_W, j_L, Pjt_array)
end


function j_propose(t, θjt, ℓjt, ℓst)
    return propose(t, θjt, ℓjt, ℓst, Pjt_array)
end

function threshold_s(t, ℓjt, ℓst)
    return threshold_m(t, ℓjt, ℓst, Pst_array)
end

function j_respond_payment(t, θjt, ℓjt, ℓst)
    return respond_payment(t, θjt, ℓjt, ℓst, Pst_array, j_W)
end

function j_respond_liq(t, θjt, ℓjt, ℓst)
    return respond_liq(t, θjt, ℓjt, ℓst, j_L, s_L)
end    

function j_respond(t, θjt, ℓjt, ℓst)
    return respond(t, θjt, ℓjt, ℓst, Pst_array, j_W, j_L, s_L)
end

j_respond (generic function with 1 method)

### Populando os arrays do jogo

#### 1,2,3 testando

In [206]:
@time begin
    
    t = T
    
    for h in 1:grid
        for l in 1:grid
            Pst(t, h, l)
            Pjt(t, h, l)
        end
    end
    
end

 14.557866 seconds (39.50 M allocations: 17.893 GiB, 12.20% gc time)


In [207]:
#testing functions
# t, hkt, lkt, lmt = T-1, 0.6, 0.4, 0.5
t, hkt, lkt, lmt = T, 60, 40, 50

(13, 60, 40, 50)

In [208]:
@btime s_propose(t, hkt, lkt, lmt)

  67.828 ns (1 allocation: 32 bytes)


(0.026000000000000023, 1.0)

In [209]:
@btime threshold_j(t, lkt, lmt)

  76.549 ns (0 allocations: 0 bytes)


91

In [210]:
@btime s_respond_payment(t, hkt, lkt, lmt)

#alocações aumentaram desde o último código, o que aconteceu?


  12.499 μs (228 allocations: 7.44 KiB)


(8.796868795624097e-10, 0.9999999200284655)

In [211]:
@btime s_respond_liq(t, hkt, lkt, lmt)

  806.742 ns (11 allocations: 1.50 KiB)


0.026000000000000023

In [212]:
@btime s_respond(t, hkt, lkt, lmt)

  13.801 μs (242 allocations: 8.97 KiB)


0.025999998800427006

In [213]:
@btime λj * s_respond(t, hkt, lkt, lmt) + (1-λj) * s_propose(t, hkt, lkt, lmt)[1]

  14.100 μs (247 allocations: 9.06 KiB)


0.02599999958494776

#### Populando penúltimo período do jogo

In [229]:
function populate_pkt(t)
    for h in 1:grid
        for l in 1:grid
            Pst(t, h, l)
            Pjt(t, h, l)
        end
    end
end

populate_pkt (generic function with 1 method)

In [220]:
@btime populate_pkt(T)

  14.410 s (39502201 allocations: 17.89 GiB)


In [230]:
function populate_wkt(t)
    for hk in 1:grid
        for lk in 1:grid
            for lm in 1:grid
                s_W[t, hk, lk, lm] = λj * s_respond(t, hk, lk, lm) + (1-λj) * s_propose(t, hk, lk, lm)[1]
                j_W[t, hk, lk, lm] = λj * j_propose(t, hk, lk, lm)[1] + (1-λj) * j_respond(t, hk, lk, lm)
            end
        end
    end
end


populate_wkt (generic function with 1 method)

In [73]:
@btime populate_wkt(T)

  36.906 s (374870201 allocations: 16.06 GiB)


### Populando todo o jogo

acho que Julia está paralelizando os loops de pkt e wkt, porém os loops de pkt precisam ser preenchidos primeiro

In [252]:
@time begin
    
    for t in T:-1:1
        @time begin
            
            populate_pkt(t)
            populate_wkt(t)
            
        end
        
    end
end


 51.000162 seconds (391.34 M allocations: 33.364 GiB, 8.75% gc time)
158.060265 seconds (1.52 G allocations: 72.846 GiB, 7.32% gc time)
170.393339 seconds (1.69 G allocations: 82.822 GiB, 7.83% gc time)
179.288165 seconds (1.79 G allocations: 85.304 GiB, 7.72% gc time)
188.488492 seconds (1.86 G allocations: 86.407 GiB, 7.50% gc time)
191.151661 seconds (1.91 G allocations: 87.494 GiB, 7.44% gc time)
194.962337 seconds (1.95 G allocations: 88.411 GiB, 7.32% gc time)
192.875956 seconds (1.98 G allocations: 89.207 GiB, 7.42% gc time)
192.383397 seconds (2.01 G allocations: 89.994 GiB, 7.52% gc time)
192.211024 seconds (2.04 G allocations: 90.943 GiB, 7.55% gc time)
196.943312 seconds (2.08 G allocations: 91.960 GiB, 7.52% gc time)
202.526867 seconds (2.12 G allocations: 93.207 GiB, 7.44% gc time)
199.144265 seconds (2.09 G allocations: 90.014 GiB, 7.35% gc time)
2309.591926 seconds (23.45 G allocations: 1.057 TiB, 7.52% gc time)


In [254]:
2309/60

38.483333333333334